# Lab 8 -- Network Layer (Control Plane)

## Introduction

This lab aims to introduce you to network addressing and routing.
In particular, you will set up and analyze a routing protocol.

For the second delivery (transport and network layer labs) submit a PDF report where you answer **only** those questions marked with a **REPORT**.

## Lab Setup

[Figure 1](#figure_1) shows the setup for this lab. 
In this lab you will be using the same topology and GNS3 project as in lab 7. The main task of this lab is to configure a routing protocol in the routers to have full connectivity between the networks.



<a id='figure_1'></a>

|<img src="figures/lab7.png" />|
|:--:| 
| *Figure 1: Lab8 Setup* |

- **Check the port forwarding and GNS3-Client configuration** and, if needed, refer back to the descriptions of lab7 to configure the enable port forwarding and GNS3.

- **Check the version of GNS3-Client and Server** and if the numbers don't match, take a look at "Update GNS3-Server" description in lab7.

- **Open GNS3 on Sahara PC**, and import the `lab7.gns3project` as a portable project and start all devices. Make sure not to open the local copy, but navigate to __File -> Import portable project__.

- **The servers** are already configured, so you don't need to do anything with them. 

  <div class="alert alert-block alert-info">
  <b>Tip:</b> To copy files between the local machine (Sahara PC) and the "ntnu_server", you can use the <code style="color: black;"> scp &lt;server_ip_or_name&gt;:&lt;remote_file_path&gt; &lt;local_path&gt;</code> command on your local machine. E.g. <code style="color: black;">scp ntnu_server:~/labs/lab7/lab7.gns3project ~/.</code> to download the GNS3 project file.
  </div>

  <div class="alert alert-block alert-info">
  <b>Tip:</b> To copy files from the containers to the "ntnu_server", you can use the <code style="color: black;">sudo cp "$(sudo find /opt/gns3/projects -name filename)" .</code> command in "ntnu_server". Change "filename" to the name of your file.
  </div>

In [ ]:
from test_lab8 import TestLab8
check_progress = TestLab8()

# Milestone 1 -- Network Addressing and Routing

## Background

In this milestone, you will learn how to set up a routing protocol. 
The IP addresses have already been set for simplicity, but the routers only know the directly connected networks. 
Thus they can not forward packets to non-directly connected networks.

- From GNS3 (with double-click or _Console_), check the routing table in the routers (__router0__, __router1__, __router2__, and __router3__) and try to understand every line in the output based on [figure 1](#figure_1).


<div style="background: #f8f8f8; overflow:auto;width:auto;border:solid gray;border-width:.1em .1em .1em .8em;padding:.2em .6em;"><pre style="margin: 0; line-height: 125%;color: black;"> ip route show 
</pre></div>


<div class="alert alert-block alert-info">
<b>Note:</b> If you are connecting to the containers with <code style="color: black;">docker attach</code>, do not forget to detach with <code style="color: black;">Ctrl+p</code> and <code style="color: black;">Ctrl+q</code>.
If you type <code style="color: black;">exit</code>, you will stop ("kill") the container. </div>

- You can try pinging the Webserver from __"router1"__ container. Do you get any response?


<div style="background: #f8f8f8; overflow:auto;width:auto;border:solid gray;border-width:.1em .1em .1em .8em;padding:.2em .6em;"><pre style="margin: 0; line-height: 125%;color: black;">
ping 129.100.1.3
</pre></div>

To have connectivity, you need to update the routing tables with the correct paths to forward packets. 
You can do this using either static or dynamic routing:

- In static routing, you manually enter routes in the routing table, but this can be a daunting task, especially in larger networks. This has been done with the `router_setup.py` in the previous lab. 

- In dynamic routing, you configure a routing protocol instructing routers to exchange information with other routers, then automatically learn routes and update the routing table.
  - In distance-vector protocols (RIP, EIGRP, BGP ... etc.), routers describe available paths (i.e., routes) to their neighbors.
  - In link-state protocols (OSPF, IS-IS, ... etc.), routers describe the state of their links to all other routers.
    
In this lab, we will use OSPF as an example, but using any other routing protocol is straightforward. [OSPF](https://en.wikipedia.org/wiki/Open_Shortest_Path_First) is a link-state routing protocol in which routers describe their link-state in a message called link-state advertisement (LSA). 
Then each router constructs a topology map of the network and calculates the shortest path using [Dijkstra's algorithm](https://en.wikipedia.org/wiki/Dijkstra%27s_algorithm)

One open-source routing software is [Free Range Routing(FRR)](https://frrouting.org/). FRR uses the Linux kernel’s routing stack for packet forwarding and implements the most commonly used protocols, such as OSPF, RIP, BGP, IS-IS, and IGRP. FRR is already installed on the routers, and the OSPF daemon is enabled by default.


## Task 1.1 -- Configure OSPF on router0

A nice feature of FRR is that it provides an interactive shell (`vtysh`) with CLI similar to Cisco IOS commands. There are different [modes](https://www.cisco.com/E-Learning/bulk/public/tac/cim/cib/using_cisco_ios_software/02_cisco_ios_hierarchy.htm) to the CLI, and certain commands are only available within a specific mode. Here is an example of how to configure OSPF on **"router0"**:


* Start the CLI by running <code>vtysh</code>. This will take you to the __EXEC mode__ (notice the change in the prompt).
* Check FRR routing table: <code>show ip route</code>. You will see only directly connected routes.

* Enter the __configuration mode__: <code>configure terminal</code> (notice the change in the prompt). This will allow you to modify the running system configuration.

* Enable OSPF routing protocol: <code>router ospf</code> (notice the change in the prompt). This will take you to  __routing configuration mode__.

* Tell OSPF which interfaces to include in its link-state advertisement (LSA). 
You only need to specify the network address, then OSPF checks which interfaces are part of this network and include them in its LSA.
Additionally, you need to specify the area:

  <div style="background: #f8f8f8; overflow:auto;width:auto;border:solid gray;border-width:.1em .1em .1em .8em;padding:.2em .6em;"><pre style="margin: 0; line-height: 125%;color: black;"><span style="color: #408080; font-style: italic">#   On the &quot;router0&quot; container:</span><br>network 129.100.1.0/29 area 0 <span style="color: #408080; font-style: italic"># ttm4200_public_net</span><br>network 129.168.1.24/29 area 0  <span style="color: #408080; font-style: italic"># link3</span><br>network 129.168.1.0/29 area 0 <span style="color: #408080; font-style: italic"># link0</span>
</pre></div>

  <div class="alert alert-block alert-info">
    <b>Note:</b> OSPF divides the network into routing areas to simplify administration and optimize traffic and resource utilization. For simplicity, you only use one area in this lab.
  </div>

    
* Exit __configuration mode__ by entering `end` or `CTRL+Z`. This will bring you back to the __EXEC mode__. Enter `exit` if you want to return to the Linux terminal.

* Check the current configuration from the CLI with <code>show running-config</code>.

<div class="alert alert-block alert-info">
<b>Tip:</b> In order to save certain configurations on the containers when using GNS3, you <b>need</b> to run the save script provided in the home directory (<b>~</b>). Use <code style="color: black;">./save.sh</code> to run it. This will save any <b>nft</b> and <b>vtysh</b> configuration. If you relaunch the GNS3 project without saving, these configurations <b>will be lost</b>. Have a look at the script to understand how to save such configurations.
</div>

<div class="alert alert-block alert-warning">
  <b>Warning:</b> When you import a GNS3 project again (i.e., after making changes and saving configurations), your previous modifications are lost because importing a project extracts a <b>fresh copy</b> from the archive, <b>discarding any changes</b> you made. Therefore, after making changes, always click <i>File -> Save Project</i> in GNS3. Then, you can open the saved version with <i>File -> Open Project</i>.
</div>

## Task 1.2 -- Configure OSPF on other routers

Your task now is to configure OSPF on __"router1"__, __"router2"__, and __"router3"__. However, **do not** include the private networks (10.20.40.0/24, 10.20.50.0/24) in the link-state advertisement.


<div class="alert alert-block alert-info">
<b>Tip:</b> If you by mistake included the private networks in the link-state advertisement, you could delete them by using, for example, <code style="color: black;">no network 10.20.40.0/24 area 0</code> in the <b>routing configuration mode</b>. </div>

   
* Ping the Webserver from __"router1"__ container and ensure you can connect to it.

In [ ]:
check_progress.test_1_1()

## Task 1.3 -- Route-Costs


* In __"router1"__, start CLI and check the FRR routing table:

  <div style="background: #f8f8f8; overflow:auto;width:auto;border:solid gray;border-width:.1em .1em .1em .8em;padding:.2em .6em;"><pre style="margin: 0; line-height: 125%; color: black;">
  vtysh 
  show ip route
</pre></div>

* Check the route towards 129.100.1.0/29, and answer the following:
  * Via which interface does the route pass?
  * What is the route cost?

In [ ]:
# Via which interface does the route pass?
route_interface =  #your answer as a string: "ether1" or "ether2"

# What is the route cost?
route_cost =  # You answer as integer

check_progress.test_1_2(route_interface, route_cost)

<div class="alert alert-block alert-info">
<b>Hint:</b> We set the Individual link cost to 100. FRR displays the information Administrative distance/Route Cost in each entry of its routing table. 
The <a href="https://docs.frrouting.org/en/latest/zebra.html#administrative-distance" target="_blank">administrative distance</a> allows FRR to decide what routes should be installed in the Linux kernel based upon the originating protocol. 
OSPF has an administrative distance of 110. </div>



## Task 1.4 -- Link Failure

* In __"router0"__, bring down the interface "ether2":


<div style="background: #f8f8f8; overflow:auto;width:auto;border:solid gray;border-width:.1em .1em .1em .8em;padding:.2em .6em;"><pre style="margin: 0; line-height: 125%; color: black;">
  sudo ip link set ether2 down 
</pre></div>


* In __"router1"__, re-check the FRR routing table again and focus on the route towards 129.100.1.0/29, then answer the following:    
  * Via which interface does the route pass after the link failure?
  * What is the route cost after the link failure?

In [ ]:
# Via which interface does the route pass after the link failure?
route_interface_after_link_failure =  #your answer as a string: "ether1" or "ether2"

# What is the route cost after the link failure?
route_cost_after_link_failure =  # Your answer as integer

check_progress.test_1_3(route_interface_after_link_failure, route_cost_after_link_failure)

<span style="font-size: 25px;">Q1. </span> **REPORT:** Based on [Figure 1](#figure_1), describe how routers detect and propagate link states through the network to build a link state database and how Dijkstra’s algorithm calculates the shortest path using this database. Then explain how the shortest path is re-calculated in case of a link failure. You can show this by highlighting the Hello and LS Update/Acknowledgement packets in screenshots from Wireshark.


<div class="alert alert-block alert-info">
<b>Hint:</b>  All OSPF routers store information about the network topology in a Link-State Database (LSDB). Hello OSPF packets are used to test reachability between neighbours, while OSPF uses Link State Advertisements (LSAs) to describe the network topology. To answer the question above, you can, for example, in "router0" bring up the interface "ether2". Then, you can start the packet capture on links between different routers, bring down and up one of the links, and observe and inspect the packets in Wireshark. You can use this command to bring down/up a link <code style="color: black;">sudo ip link set [interface_name] down/up</code>. </div>

<div class="alert alert-block alert-info">
<b>Note:</b> If the Wireshark window fails to appear when you "<i>Start capture</i>" in GNS3, locate <i>Edit -> Preferences -> Packet capture</i>. Under <i>Packet capture reader command</i> delete <b>--capture-comment "{project} {link_description}"</b>. Then stop and start the capture again. </div>


## Optional Task 1.5 -- Another Routing Protocol

* Configure other routing protocols, such as RIP, EIGRP, IS-IS, or BGP. For example, to configure RIP on "router0":
  *  Enable the RIP daemon in "/etc/frr/daemons" by replacing "ripd=no" to "ripd=yes". Since OSPF has a lower administrative distance than RIP, you need to disable it. Otherwise, OSPF routes will take precedence.
  * Restart FRR: <code>sudo service frr restart</code>.
  * Configure RIP using the CLI:


<div style="background: #f8f8f8; overflow:auto;width:auto;border:solid gray;border-width:.1em .1em .1em .8em;padding:.2em .6em;"><pre style="margin: 0; line-height: 125%;color: black;"><span style="color: #408080; font-style: italic">#   On the &quot;router0&quot; container:</span>
vtysh
configure terminal
router rip
network 129.100.1.0/29
network 129.168.1.24/29
network 129.168.1.0/29
end
exit
</pre></div>
